# WS4Her 2021.1 - Gabarito ELIZA (Aplicação) 👩💻

Gabarito da parte de aplicação do Workshop for Her organizado pelo Turing USP no primeiro semestre de 2021, no qual implementamos um chatbot simples inpirado no chatbot ELIZA.

### Importando módulos

In [ ]:
import re # regex para fazer matching de padrões no texto
import random # para numeros e selecao aleatorios

### Definindo constantes

Nossa ELIZA irá conversar com o usuário sobre seus sentimentos.

Assim, implementamos ela de modo que ela identifique se esse usuário está falando sobre algum sentimento/sensação ou fazendo um desabafo.

Definimos nessa primeira parte respostas para cada uma desas situações, além de respostas genéricas para fazê-lo falar mais.

In [ ]:
RESPOSTAS_GENERICAS = ['Me fale mais sobre isso.', 'Interessante... elabore.',
                       'O que isso te lembra?']

RESPOSTAS_SENSACAO = ['Por que você está {}?', 'Para você, como é se sentir {}?',
                      'Por que você está {}?']

REPOSTAS_DESABAFO = ['O que você pensa sobre {}?', 'Para você, o que é {}?',
                     'Como você se sente em relação a {}?']

STOPWORDS = ['eu', 'não', 'quero', 'acho', 'e', 'você', 'nós', 'acredito', 'o',
             'a', 'os', 'um', 'uns', 'umas', 'as', 'uma', 'me', 'te', 'lhe',
             'nos', 'n', 'pq', 'porque']

### Funções

Para que seja mais fácil pensar, vamos separar nosso programa em blocos usando funções. Assim, resolvemos um problema menor por vez e então juntamos as soluções.

Nessa primeira função, vamos determinar a intenção de uma frase, ou seja, se é uma sensação ou um desabafo.

Para identificar uma sensação, vamos usar regex com o módulo re. O padrão em `padrao_sensacao` identifica se a frase começa com "eu estou"/"eu tô"/"to" e outras variações dessa expressão.

`re.search()` identifica esse padrão na frase recebida: se sim, ela devolve um objeto de match; se não, retorna não.

Um objeto de match tem valor `True`, enquanto `None` tem valor `False`. Assim, podemos usá-los em expressões condicionais.

In [ ]:
def determina_intencao(text):
  '''
  Funcao que recebe um texto (string), verifica se ha um padrão
  que indica sensação. Se houver, ele retorna uma string
  'sensacao', se não, retorna 'desabafo'.
  '''
  
  # padrao para identificar 'eu estou' e derivados
  padrao_sensacao = "[Ee]u ([Tt][oô]|estou) |[Tt][oô] |estou "

  # fazer o match do padrao na frase
  match = re.search(padrao_sensacao, text) # retorna None se nao houver match

  if match:
    # se houver match, intencao recebe 'sensacao'
    intencao = 'sensacao'
  else:
    # se nao houver, intencao recebe 'desabafo'
    intencao = 'desabafo'
 
  return intencao

Agora, fazemos uma função que responde uma mensagem contendo uma sensação.

`re.search(padrao_sensacao, text).group(0)` retorna o trecho na frase que deu match com o padrão.

`"texto".find("trecho")` retorna o índice da primeira ocorrência do trecho no texto.

`random.choice()` recebe uma lista e retorna um item aleatório desta.

`resposta.format("trecho")` preenche {} na resposta com "trecho".

In [ ]:
def responde_sensacao(text):
  '''
  Funcao que recebe um texto que possua "eu estou" ou variações,
  armazena o resto da frase, seleciona uma resposta aleatoria
  das lista RESPOSAS_SENSACAO e retorna essa resposta com o
  trecho armazenado inserido.
  '''

  padrao_sensacao = "[Ee]u ([Tt][oô]|estou) |[Tt][oô] |estou "
  # sensacao recebe a parte do texto que deu match com o padrao
  sensacao = re.search(padrao_sensacao, text).group(0)

  # resto_frase recebe o trecho que vem depois da parte que deu match
  resto_frase = text[text.find(sensacao)+len(sensacao):]

  # escolhe uma resposta da lista RESPOSTAS_SENSACAO aleatoriamente
  resposta = random.choice(RESPOSTAS_SENSACAO)

  # retorna a resposta escolhida com resto_frase no lugar de {}
  return resposta.format(resto_frase)

Agora, a função que responde um desabafo.

`text.split()` separa uma string em palavras e retorna uma lista com elas

`string.lower()` deixa o texto da string em letras todas minúsculas.

In [ ]:
def responde_desabafo(text):
  '''
  Funcao que recebe um texto de desabafo, seleciona uma resposta
  aleatoria de RESPOSTAS_DESABAFO, completa ela com
  uma palavra significativa do texto e retorna essa resposta.
  Caso nao haja palavras significativas no texto, a resposta
  sera 'Hm...'.
  '''

  # escolhe uma resposta da lista RESPOSTAS_DESABAFO aleatoriamente
  resposta = random.choice(REPOSTAS_DESABAFO)

  # cria uma lista com cada palavra no texto
  lista_de_palavras = text.split()

  # cria uma lista vazia
  lista_de_palavras_minusculas_validas = []

  # vamos percorrer a lista_de_palavras
  for palavra in lista_de_palavras:

    # deixa a palavra toda em letras minusculas
    palavra = palavra.lower()
    
    # se a palavra nao estiver em STOPWORDS, adiciona na lista_de_palavras_minusculas_validas
    if palavra not in STOPWORDS:
      lista_de_palavras_minusculas_validas.append(palavra)

  num_palavras_validas = len(lista_de_palavras_minusculas_validas)
  if num_palavras_validas > 1:
    # se houver mais de uma palavra na lista_de_palavras_minusculas_validas
    # escolhe uma palavra aleatoria dessa lista e armazena em tema
    tema = random.choice(lista_de_palavras_minusculas_validas)
    # e entao insere tema na resposta
    resposta = resposta.format(tema)

  elif num_palavras_validas == 1:
    # caso haja apenas uma palavra na lista, ela sera inserida na resposta
    tema = lista_de_palavras_minusculas_validas[0]
    resposta = resposta.format(tema)

  elif num_palavras_validas == 0:
    # se nao ha palavras na lista, a resposta sera 'hm...'
    resposta = 'Hm...'

  return resposta

Agora, definimos a função `responder`, que irá identificar a intenção e retornar a resposta.

Além das repostas de sensação e desabafo, há também a chance da Eliza responder com uma resposta genérica. Vamos estabelecer que isso ocorre com probabilidade 30%, ou seja, 3 em cada 10 vezes.

`random.random()` gera um número real entre 0 e 1.

In [ ]:
def responder(mensagem):
  '''
  Funcao que recebe uma mensagem, determina sua intencao e retorna uma
  resposta de acordo com essa intencao ou responde genericamente com
  probabilidade de 30%.
  '''

  # gera um float aleatorio entre 0 e 1
  numero_aleatorizador  = random.random()

  # se esse numero for menor que 0.3,
  # a resposta sera aleatoriamente escolhida entre as RESPOSTAS_GENERICAS
  if numero_aleatorizador < 0.3:
    resposta = random.choice(RESPOSTAS_GENERICAS)

  # caso contrario, determina a intencao da mensagem
  # e atribui uma resposta de acordo com essa intencao
  else:
    intencao = determina_intencao(mensagem)

    if intencao == 'desabafo':
      resposta = responde_desabafo(mensagem)

    elif intencao == 'sensacao':
      resposta = responde_sensacao(mensagem)

  return resposta

### Juntando tudo

In [ ]:
def main():
  # a primeira mensagem da Eliza sera uma saudacao
  print("Olá! Eu sou a Eliza.\nComo você está se sentindo hoje? Sobre o que gostaria de falar?")

  # loop que se repetira 6 vezes
  for i in range(6):
    # recebe uma mensagem do usuario
    mensagem = input()

    # gera uma resposta
    resposta = responder(mensagem)

    # printa a resposta
    print(resposta)

  # recebe ultima mensagem do usuario
  mensagem = input()

### Testando

In [ ]:
main()

Olá! Eu sou a Eliza.
Como você está se sentindo hoje? Sobre o que gostaria de falar?
Nossa mas que pergunta estou pensando
Por que você está pensando?
Você tá filosófica hein eliza
O que você pensa sobre filosófica?
tem que pensar mt pra ser filosofica ne
O que isso te lembra?
lembra minha amiga eliza filosofando agora
O que você pensa sobre minha?
é sua
Como você se sente em relação a sua?
Suar? Não curto
Me fale mais sobre isso.
Quando a gente sua a gente fede né, eu pessoalmente não gosto.
